In [1]:
import setup_django
setup_django.init()

PWD environment variable is not set. Setting it to: ..


In [38]:
from ratings.models import Rating
import pandas as pd
from django.conf import settings
from anime.models import Anime
ratings_path = settings.DATA_DIR / "reviews.csv" #pathlib
ratings_path.exists()

True

In [3]:
qs = Rating.objects.all()
missing_anime_ids = []
for obj in qs:
    if obj.content_object is None:
        missing_anime_ids.append(obj.object_id)

_total = len(missing_anime_ids)
total_missing = list(set(missing_anime_ids))
print(len(total_missing), _total)

3436 169244


In [4]:
df = pd.read_csv(ratings_path)
df.head()

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149


In [10]:
ms_df = df.copy()[df['anime_uid'].isin(total_missing)]
ms_df = ms_df.drop_duplicates(subset=['anime_uid'])
ms_df.head()

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664
6,247454,Waffle_Empress,16664,\n \n \n \n ...,6,"{'Overall': '6', 'Story': '6', 'Animation': '9...",https://myanimelist.net/reviews.php?id=247454
16,262977,OVERPOWERED99,30276,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '7', 'Animation': '1...",https://myanimelist.net/reviews.php?id=262977


In [12]:
ms_df.shape[0] == len(total_missing)

True

In [15]:
anime = settings.DATA_DIR / "animes.csv" #pathlib
anime.exists()

True

In [16]:
anime_df = pd.read_csv(anime)
anime_df.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...


In [21]:
# This assumes you've not renamed columns for consistency; adjust column names as needed
ms_df['exists_in_csv'] = ms_df['anime_uid'].isin(anime_df['uid'])
ms_df.head()
ms_df.shape

(3436, 8)

In [96]:
from ratings.models import Rating
from anime.models import Anime
from django.contrib.contenttypes.models import ContentType

# Get the ContentType for the Anime model
anime_content_type = ContentType.objects.get_for_model(Anime)

for uid in total_missing:
    # Assuming `object_id` is a field in the Rating model that stores the ID of the related object
    # and `content_type` is used to indicate the model of the related object
    Rating.objects.filter(object_id=uid, content_type=anime_content_type).delete()

    # Then, delete the Anime object using `id` instead of `uid`
    try:
        anime = Anime.objects.get(id=uid)  # Use `id` here instead of `uid`
        anime.delete()
    except Anime.DoesNotExist:
        print(f"No Anime found with id={uid}.")

No Anime found with id=16385.
No Anime found with id=32772.
No Anime found with id=16389.
No Anime found with id=16395.
No Anime found with id=16397.
No Anime found with id=24591.
No Anime found with id=32785.
No Anime found with id=16405.
No Anime found with id=32792.
No Anime found with id=32801.
No Anime found with id=16417.
No Anime found with id=16419.
No Anime found with id=32804.
No Anime found with id=32805.
No Anime found with id=32813.
No Anime found with id=24625.
No Anime found with id=16434.
No Anime found with id=24627.
No Anime found with id=16436.
No Anime found with id=24629.
No Anime found with id=16442.
No Anime found with id=32827.
No Anime found with id=16444.
No Anime found with id=32828.
No Anime found with id=32830.
No Anime found with id=32829.
No Anime found with id=32833.
No Anime found with id=24641.
No Anime found with id=32836.
No Anime found with id=35835.
No Anime found with id=32843.
No Anime found with id=24655.
No Anime found with id=32851.
No Anime f

In [99]:
print(total_missing)

[16385, 32772, 16389, 16395, 16397, 24591, 32785, 16405, 32792, 32801, 16417, 16419, 32804, 32805, 32813, 24625, 16434, 24627, 16436, 24629, 16442, 32827, 16444, 32828, 32830, 32829, 32833, 24641, 32836, 35835, 32843, 24655, 32851, 16468, 32852, 32854, 16472, 16474, 16476, 32863, 32864, 32866, 32867, 32869, 32870, 32871, 32872, 32878, 24687, 32881, 16498, 32887, 24695, 24699, 16508, 24701, 32894, 32895, 16512, 24705, 24703, 32899, 32900, 32901, 16518, 32902, 32906, 16524, 16528, 24727, 32924, 32930, 32935, 32936, 32937, 24745, 24751, 32946, 32947, 32948, 32949, 32950, 24765, 32959, 16576, 32961, 32962, 24781, 16592, 32977, 32979, 32981, 32982, 32983, 16602, 16604, 16606, 16608, 32993, 16610, 32995, 32997, 32998, 33003, 33004, 24817, 33010, 33011, 33012, 24821, 33013, 24823, 33018, 33019, 33021, 16638, 33023, 24833, 33025, 24835, 16642, 16644, 33027, 33028, 33032, 33026, 33034, 33035, 33036, 33037, 33031, 16656, 24849, 33042, 24851, 33041, 33045, 16662, 33047, 33046, 33049, 16664, 33051

In [106]:
from anime.models import Anime

# Replace 123 with an actual id from your total_missing list
try:
    anime = Anime.objects.get(id=16355)
    print(anime)
except Anime.DoesNotExist:
    print("No Anime found with id=16355")


No Anime found with id=16355


In [109]:
!python manage.py makemigrations
!python manage.py migrate


No changes detected
Operations to perform:
  Apply all migrations: account, admin, anime, auth, contenttypes, django_celery_beat, django_celery_results, exports, ratings, sessions, sites, socialaccount
Running migrations:
  No migrations to apply.
